In [95]:
import os 

In [96]:
%pwd

'C:\\ML_OPS\\Projects\\project1'

In [97]:
#os.chdir('C:\ML_OPS\Projects\project1')

In [98]:
from src.DataScienceProject.config import *
from src.DataScienceProject import logger
from dataclasses import dataclass
from pathlib import Path
from src.DataScienceProject.constants import *


In [99]:
from src.DataScienceProject.utils.shared import read_yaml,create_directories

In [100]:
@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: str
    l1_ratio: str
    target_column: str
    

In [101]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])
    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_trainer_config= ModelTrainingConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = float(params.alpha),
            l1_ratio= float(params.l1ratio),
            target_column = schema.name
        )
        return model_trainer_config

        

In [102]:
import pandas as pd 
from sklearn.linear_model import ElasticNet
import joblib # Alternative of pickle


In [103]:
class ModelTrainer:
    def __init__(self,config: ModelTrainingConfig):
        self.config = config
        # self.model = ElasticNet(alpha=config.alpha, l1_ratio=config.l1_ratio)
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(columns=[self.config.target_column],axis=1)
        test_x = test_data.drop(columns=[self.config.target_column],axis=1)

        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio)
        lr.fit(train_x, train_y)
        print("Train Path Exists:", os.path.exists(self.config.train_data_path))
        print("Test Path Exists:", os.path.exists(self.config.test_data_path))

        logger.info('Model fit training done.')


        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

In [104]:
create_directories([Path("artifacts/test_folder")])

[2025-06-06 12:33:23,173 : INFO : shared : created directory at : artifacts\test_folder]


In [105]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_trainer_config()
    trainer = ModelTrainer(config=model_config)
    trainer.train()
except Exception as e:
    print("Model Trainer Error:", e)


[2025-06-06 12:33:23,219 : INFO : shared : Yaml File: config\config.yaml loaded successfully!]
[2025-06-06 12:33:23,250 : INFO : shared : Yaml File: params.yaml loaded successfully!]
[2025-06-06 12:33:23,258 : INFO : shared : Yaml File: schema.yaml loaded successfully!]
[2025-06-06 12:33:23,263 : INFO : shared : created directory at : artifacts]
[2025-06-06 12:33:23,265 : INFO : shared : created directory at : artifacts/model_trainer]
Train Path Exists: True
Test Path Exists: True
[2025-06-06 12:33:23,440 : INFO : 3023829112 : Model fit training done.]
